In [35]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta
from tableauscraper import TableauScraper as TS

### Download the tableau dashboard

In [36]:
url = "https://public.tableau.com/views/CaliforniaUrbanWaterProduction_15785959527960/StatewideUseandProduction"

In [37]:
ts = TS()
ts.loads(url)
workbook = ts.getWorkbook()

In [38]:
for t in workbook.worksheets:
    if t.name == "RGPCD_HYDRO":
        total_hydro_df = t.data
    elif t.name == "Total_Production":
        total_df = t.data
    else:
        pass

In [39]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [40]:
ws = ts.getWorksheet("RGPCD_HYDRO")

In [41]:
df_list = []
for m in months:
    wb = ws.setFilter('MONTH(Reporting Month)', m)
    monthlyWs = wb.getWorksheet('RGPCD_HYDRO')
    monthly_df = monthlyWs.data
    monthly_df["month"] = m
    df_list.append(monthly_df)

2022-06-01 10:56:25,227 - tableauScraper - WARNING - no data dictionary present in response
2022-06-01 10:56:25,227 - tableauScraper - WARNING - no data dictionary present in response
2022-06-01 10:56:25,227 - tableauScraper - WARNING - no data dictionary present in response
2022-06-01 10:56:32,107 - tableauScraper - WARNING - no data dictionary present in response
2022-06-01 10:56:32,107 - tableauScraper - WARNING - no data dictionary present in response
2022-06-01 10:56:32,107 - tableauScraper - WARNING - no data dictionary present in response


In [42]:
df = pd.concat(df_list)

In [43]:
pivot = df.pivot(index=['Region-value','month'], columns='YEAR(Reporting Month)-alias', values='AGG(Pop_Weighted_RGPCD)-alias').reset_index()

In [44]:
trim_cols = ['Region-value', 'AGG(NumReporting)-alias', 'month',
       'YEAR(Reporting Month)-alias',
       'AGG(Pop_Weighted_RGPCD)-alias']

In [45]:
df_trim = df[trim_cols]

In [46]:
df_rename = df_trim.rename(columns={
    'Region-value':'hydrologic_region', 
    'AGG(NumReporting)-alias': 'agencies_reporting', 
    'YEAR(Reporting Month)-alias': 'year',
    'AGG(Pop_Weighted_RGPCD)-alias': 'pop_weighted_rgpcd'
})

In [47]:
df_rename.loc[(df_rename.hydrologic_region == "%all%"), "hydrologic_region"] = "Statewide"

In [48]:
df_rename.to_csv("../data/raw/regional-usage.csv", index=False)